# AirLens Policy Effect Analysis

**PRD §6 기반 — Before/After ΔPM2.5 + 지역 그룹 비교**

참고 논문:
- Heffernan et al. (2024) ML 기반 CITS 정책 효과 추정
- MAIAC AOD + MLR 한국 연구 (R²~0.64, RMSE~9 µg/m³)


In [ ]:
import json, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path

BASE = Path('../app/data')
with open(BASE / 'policy-analytics.json') as f:
    analytics = json.load(f)

df = pd.DataFrame(analytics['policies'])
region_avg = analytics['regionAvgPct']
global_avg = analytics['globalAvgPct']
print(f'Total policies: {len(df)}, Countries: {df["country"].nunique()}')
print(f'Global avg change: {global_avg}%')
df.head(3)

## 1. 전체 정책 Before → After 분포

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.hist(df['pct'], bins=25, color='#25e2f4', edgecolor='white', alpha=0.85)
ax1.axvline(global_avg, color='#ef4444', linestyle='--', lw=2, label=f'Global avg: {global_avg}%')
ax1.axvline(0, color='gray', linestyle=':', lw=1)
ax1.set_xlabel('PM2.5 Change (%)')
ax1.set_ylabel('# Policies')
ax1.set_title('Distribution of PM2.5 Change After Policy', fontweight='bold')
ax1.legend()

ax2 = axes[1]
colors = ['#10b981' if p < -10 else '#f59e0b' if p < 0 else '#ef4444' for p in df['pct']]
ax2.scatter(df['before'], df['after'], c=colors, alpha=0.7, s=50, edgecolors='white', lw=0.5)
lim = max(df['before'].max(), df['after'].max()) + 5
ax2.plot([0, lim], [0, lim], 'gray', linestyle=':', lw=1, label='No change')
ax2.set_xlabel('Before PM2.5 (µg/m³)')
ax2.set_ylabel('After PM2.5 (µg/m³)')
ax2.set_title('Before vs After PM2.5 by Policy', fontweight='bold')
patches = [mpatches.Patch(color='#10b981', label='> -10%'),
           mpatches.Patch(color='#f59e0b', label='0 ~ -10%'),
           mpatches.Patch(color='#ef4444', label='Increased')]
ax2.legend(handles=patches, fontsize=9)
plt.tight_layout()
Path('../docs').mkdir(exist_ok=True)
plt.savefig('../docs/policy_effect_overview.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. 지역별 그룹 평균 비교 (PRD §6.2)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))
regions = sorted(region_avg.items(), key=lambda x: x[1])
labels  = [r[0] for r in regions]
values  = [r[1] for r in regions]
bar_colors = ['#10b981' if v < -15 else '#25e2f4' if v < -5 else '#f59e0b' for v in values]
bars = ax.barh(labels, values, color=bar_colors, edgecolor='white', alpha=0.9)
ax.axvline(global_avg, color='#ef4444', linestyle='--', lw=2, label=f'Global avg ({global_avg}%)')
ax.axvline(0, color='gray', linestyle=':', lw=1)
for bar, v in zip(bars, values):
    ax.text(v-0.3, bar.get_y()+bar.get_height()/2, f'{v}%',
            va='center', ha='right', fontsize=9, color='white', fontweight='bold')
ax.set_xlabel('Average PM2.5 Change (%)')
ax.set_title('Regional Average PM2.5 Change After Policy', fontweight='bold')
ax.legend()
plt.tight_layout()
plt.savefig('../docs/region_group_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. 이벤트 스터디 — 한국 정책 Before/After

In [ ]:
with open(BASE / 'policy-impact/south-korea.json') as f:
    kr = json.load(f)

policies = kr['policies']
fig, axes = plt.subplots(1, len(policies), figsize=(14, 5), sharey=True)
if len(policies) == 1:
    axes = [axes]

for ax, policy in zip(axes, policies):
    imp = policy.get('impact', {})
    timeline = policy.get('timeline', [])
    if not timeline:
        continue
    years = [t['date'][:4] for t in timeline]
    vals  = [t['pm25'] for t in timeline]
    py    = policy['implementationDate'][:4]
    before = imp.get('beforePeriod', {}).get('meanPM25')
    after  = imp.get('afterPeriod',  {}).get('meanPM25')
    pct    = imp.get('analysis',     {}).get('percentChange', 0)
    ax.plot(years, vals, marker='o', color='#25e2f4', lw=2, markersize=6)
    ax.axvline(py, color='#ef4444', linestyle='--', lw=2, label=f'Enacted ({py})')
    if before:
        ax.axhline(before, color='#94a3b8', linestyle=':', lw=1.5, label=f'Before: {before}')
    if after:
        ax.axhline(after, color='#10b981', linestyle=':', lw=1.5, label=f'After: {after}')
    sign = '' if pct < 0 else '+'
    ax.set_title(f"{policy['name'][:30]}\n{sign}{pct:.1f}%", fontsize=9, fontweight='bold')
    ax.set_ylabel('PM2.5 µg/m³')
    ax.tick_params(axis='x', rotation=45)
    ax.legend(fontsize=8)

fig.suptitle('South Korea — Policy Event Study', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../docs/korea_event_study.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. 상대 효과 Top 10 (지역 평균 대비)

In [ ]:
df['region_avg']      = df['region'].map(region_avg)
df['relative_effect'] = df['pct'] - df['region_avg']

top10 = df.nsmallest(10, 'relative_effect')
labels = [f"{r.country}\n{r.policyName[:22]}..." for r in top10.itertuples()]
colors_r = ['#10b981' if v < -5 else '#25e2f4' for v in top10['relative_effect']]

fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(labels, top10['relative_effect'], color=colors_r, alpha=0.9, edgecolor='white')
ax.axvline(0, color='gray', linestyle=':', lw=1)
ax.set_xlabel('Relative Effect vs Region Average (%p)')
ax.set_title('Top 10 — Outperforming Regional Average', fontweight='bold')
plt.tight_layout()
plt.savefig('../docs/relative_effect_top10.png', dpi=150, bbox_inches='tight')
plt.show()
print(top10[['country','policyName','pct','region_avg','relative_effect']].to_string(index=False))

## 5. AOD-MLR 민감도 분석 (PRD §5.1)

In [ ]:
COEF = {'b0':5.2,'b1':40.0,'b2':-0.003,'b3':0.18,'b4':-0.20,'b5':-0.005}
aod_range = np.linspace(0.05, 1.5, 50)
pbl_vals  = [500, 800, 1200]
rh, temp, elev = 65, 15, 50

fig, ax = plt.subplots(figsize=(10, 5))
for pbl in pbl_vals:
    pm = (COEF['b0'] + COEF['b1']*aod_range + COEF['b2']*pbl +
          COEF['b3']*rh + COEF['b4']*temp + COEF['b5']*elev)
    ax.plot(aod_range, pm, label=f'PBL={pbl}m', lw=2)
ax.axhline(35, color='#f59e0b', linestyle='--', lw=1.5, alpha=0.7, label='Moderate (35)')
ax.axhline(55, color='#ef4444', linestyle='--', lw=1.5, alpha=0.7, label='Unhealthy (55)')
ax.set_xlabel('AOD')
ax.set_ylabel('Estimated PM2.5 (µg/m³)')
ax.set_title('AOD → PM2.5 MLR Sensitivity (RH=65%, T=15°C)', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../docs/aod_mlr_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()
print('MLR coefficients:', COEF)
print('Reference: MAIAC AOD Korea study R²~0.64, RMSE~9 µg/m³')

## 요약

| 분석 | 결과 |
|------|------|
| 전체 정책 수 | 132개 (66개국) |
| 글로벌 평균 PM2.5 변화율 | −17.2% |
| 가장 효과 큰 지역 | Northern Europe (−33.9%) |
| 가장 적은 지역 | South Asia (−2.2%) |

생성 파일:
- `docs/policy_effect_overview.png`
- `docs/region_group_comparison.png`
- `docs/korea_event_study.png`
- `docs/relative_effect_top10.png`
- `docs/aod_mlr_sensitivity.png`
